# Alumno: Gerardo de Miguel González

## Predicción Keras 

**Practica 2** - predecir la precipitación en Madrid. Hay que entregar tanto el archivo con la prediccion (.rda) como el codigo que lo genera (.html a ser posible, si lo haceis en jupyter notebooks lo podeis guardar como .html).

 1) Si habéis predicho la ocurrencia de precipitacion y la cantidad de precipitacion por separado entonces enviadme ambas predicciones tal que:

```r
save(yTest_ocu,yTest_reg,file="yTest.rda")
```

 2) Si habeis predicho la ocurrencia y la cantidad de precipitation con el mismo modelo entonces enviadmelo asi:

```r
save(yTest,file="yTest.rda")
```

## Descripción detallada

El objetivo es diseñar una red neuronal que sea capaz de predecir el valor de la precipitación mediante la información de variables de larga escala. El dataset consiste en 450 predictores y un predictando, la precipitación en Madrid.

Hay que tener en cuenta que la precipitación es una variable mixta, ya que habrá días con un valor exacto de 0 y otros que se encuentren en el intervalo (0,inf).

Así pues, diseñar una/s red neuronal/es con KERAS, que sea capaz de predecir la precipitación de acuerdo con los siguientes índices:

  - La ocurrencia de precipitación se evalúa con el AUC.
  - La cantidad de precipitación se evalúa con el MSE.

Podréis valeros de todas las técnicas aprendidas hasta ahora: early stopping, regularización, backpropagation + momento, ajustar el learning rate, poner varias capas y demás. Esto no quiere decir que la red que contenga toda esta variedad de técnicas será la que obtenga un menor error. Tenéis que jugar y probar distintos diseños de redes con el fin de encontrar la que consideréis que obtendría un menor error en un dataset distinto (es decir, que tenga buena
capacidad de generalización).

Construiréis vuestro modelo usando xTrain e yTrain, y después evaluaréis el modelo en xTest. La predicción que resulte al evaluar el modelo en xTest es lo que tenéis que guardar en un fichero y lo que se envía para corrección:

```r
save(yTest, file = “yTest.rda”)
```

**NOTA**: se considera día de lluvia si el valor de precipitación es mayor que 1, sino convertir a 0.

## Dataset Meteo Madrid

Se proporciona el dataset `Madrid_Alumno.rda` junto con el enunciado de la práctica

In [3]:
file.info('data/Madrid_Alumno.rda')

,size,isdir,mode,mtime,ctime,atime,uid,gid,uname,grname
data/Madrid_Alumno.rda,20099579,FALSE,644,2019-03-22 11:31:26,2019-03-22 11:31:26,2019-03-22 11:35:08,1000,100,jovyan,users


In [4]:
#::GMG::Cargo en memoria el rda
# https://stat.ethz.ch/R-manual/R-devel/library/base/html/load.html
load('data/Madrid_Alumno.rda')

In [5]:
#::GMG::Tengo los tres "objetos" que se mencionan en el enunciado
ls()

[1] "xTest"  "xTrain" "yTrain"

In [6]:
str(xTest)

 num [1:2192, 1:450] 57168 56932 56256 55960 55665 ...


In [12]:
class(xTest)

[1] "matrix"

In [11]:
dim(xTest)

[1] 2192  450

In [7]:
str(xTrain)

 num [1:8766, 1:450] 57043 56963 56523 54628 53584 ...


In [13]:
class(xTrain)

[1] "matrix"

In [10]:
dim(xTrain)

[1] 8766  450

In [8]:
str(yTrain)

 num [1:8766(1d)] 0.2 0 7.4 0 0 1.6 7.8 0 0 0 ...
 - attr(*, "dimensions")= chr "time"


In [14]:
class(yTrain)

[1] "array"

In [9]:
dim(yTrain)

[1] 8766

In [15]:
dim(as.matrix(yTrain))

[1] 8766    1

## Biblioteca

In [1]:
#::GMG::Hay que usar Keras en R
library(keras)

## Preparación de los datos

In [ ]:
#::GMG:: Normalzación


## Modelo de Clasificación

### Construcción de la red inicial

In [22]:
#::GMG::Sigo el guión que hemos trabajado en clase
# Paso 1. Defino mi modelo como una estructura secuencial.
modelo_class <- keras_model_sequential()

In [23]:
#::GMG::Comienzo con una red "simple" con una capa oculta de 10 neuronas
# Paso 2. Defino la topología de la red neuronal
# 450 -- 5 -- 1
modelo_class %>% 
  # capa oculta
  layer_dense(units = 10, input_shape = 450, activation = "sigmoid") %>% 
  layer_dense(units = 1 , activation = "sigmoid") 

In [24]:
#::GMG::Visualizo mi modelo
str(modelo_class)

Model
________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
dense_5 (Dense)                     (None, 10)                      4510        
________________________________________________________________________________
dense_6 (Dense)                     (None, 1)                       11          
Total params: 4,521
Trainable params: 4,521
Non-trainable params: 0
________________________________________________________________________________




In [ ]:
# una medida mejor que accuracy cuando hay desequlibrio entre clases
# Fmeasure <- 2 * precision * recall / (precision + recall)

In [ ]:
# ::GMG:: Me mantengo en lo clásico: backprop con SGD, de momento sin optimizaciones
# Paso 3. Defino cómo va a aprender la red: algoritmo de aprendizaje y función a minimizar
modelo_class %>% compile(
  optimizer = optimizer_sgd(lr = 0.1),
  loss='binary_crossentropy',
  metrics=['accuracy']
)

### Entrenamiento

In [ ]:
# Paso 4. Entreno el modelo y lo valido
# ::GMG:: Voy a controlar un sobreajuste separando un validation set
#         en el entrenamiento (tal y como nos han indicado en las sesiones prácticas)
#::nota:: además voy a monitorizar el tiempo de entrenamiento
timeKERAS_begin <- Sys.time(
history <- modelo_class %>% fit(x_train, 
                                y_train, 
                                epochs = 20, 
                                batch_size = 100,
                                validation_split = 0.2)
timeKERAS_end <- Sys.time()
timeKERAS <- timeKERAS_end - timeKERAS_begin
timeKERAS

In [ ]:
#::GMG::Veo el resultado
plot(history)

In [ ]:
# Paso 5. (opcional) regularización L1, L2, ElasticNet(L1/L2)


In [ ]:
# Paso X: evaluación final el modelo
# ::GMG::El problema que tenemos aquí es que no tenemos el yTest
#        por lo tanto este sería el paso previo a la grabación del
#        resultado que nos piden
#pred <- predict(modelo_class,x_test)

### Tuning de hiperparametros

## Modelo de Regresión

### Construcción de la red inicial

### Entrenamiento

### Tuning de hiperparametros

## Grabación de modelos